# 🚦 Acidentes de Trânsito em Nova Iorque: Continuação Exploratória

> *Este projeto dá continuidade ao MVP anterior desenvolvido com foco em aprendizado de máquina, intitulado "Predição de Gravidade em Acidentes de Trânsito em NY" ([GitHub](https://github.com/RobertoSacoman/Estudos/blob/main/MVP_ML_Acidentes_Transito_NY.ipynb)). Agora, a abordagem se expande com um pipeline de dados robusto, estruturado segundo princípios de Data Engineering e Data Warehousing.*

---

<img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/acidente%20de%20transito%20-%20NY.jpg" alt="Acidente de Trânsito em NY" width="1200"/>

## 📍 Introdução

Com mais de 8 milhões de habitantes e um fluxo diário intenso de veículos, Nova Iorque é palco de uma das malhas urbanas mais desafiadoras do mundo. O trânsito, além de ser essencial para o funcionamento da cidade, representa um constante risco à vida, à infraestrutura e à mobilidade urbana.

O objetivo deste estudo é explorar, tratar e modelar dados de acidentes de trânsito em NY, utilizando tecnologias de Big Data e arquiteturas de Data Lake/Data Warehouse, em um processo completo de ETL, análise da qualidade dos dados e construção de modelo dimensional.

---

## 📊 Por que estudar os Acidentes em NY?

A segurança viária é um tema crítico em áreas metropolitanas, e em Nova Iorque os desafios se destacam por fatores como:

- **Alta densidade de tráfego**: congestionamentos frequentes, especialmente em cruzamentos movimentados;
- **Comportamentos arriscados**: uso de celular ao volante, direção sob efeito de álcool, excesso de velocidade;
- **Diversidade modal**: ciclistas, pedestres, motoristas e veículos de entrega disputam o mesmo espaço urbano;
- **Influência do clima**: chuva, gelo e neve impactam diretamente na ocorrência de colisões.

Esses fatores tornam Nova Iorque um **laboratório real para análise de dados complexos e multivariados** no contexto urbano.

---

## 🎯 Objetivo deste MVP

Este projeto busca:

1. Realizar a ingestão e o tratamento de um conjunto robusto de dados públicos sobre colisões;
2. Conduzir uma análise exploratória com foco em qualidade, padronização e enriquecimento dos dados;
3. Modelar os dados em estrutura estrela ou flat, facilitando visualizações e análises futuras;
4. Identificar fatores-chave de risco e padrões de comportamento urbano relacionados a acidentes.

---

## 🧬 Diferenciais em relação ao MVP anterior

Enquanto o MVP anterior teve como foco **a modelagem preditiva com Machine Learning**, este trabalho se concentra em:

- Engenharia de dados e pipelines
- Qualidade e preparação de dados para relatórios e dashboards
- Estruturação analítica com **catálogo de dados e modelo dimensional**
- Apoio a decisões estratégicas com base em dados confiáveis

---

> “Dados mal estruturados geram decisões mal direcionadas. Com este projeto, busco transformar informação bruta em insights confiáveis e aplicáveis à segurança urbana de NY.”


## 🧬 Linhagem dos Dados

A seguir, é apresentada a linhagem dos dados deste projeto, descrevendo a trajetória desde a origem até a estruturação na camada Silver.

---

### 📍 Origem dos Dados

- **Fonte**: Dataset público sobre colisões de trânsito na cidade de Nova Iorque (NYC Open Data).
- **Formato**: Arquivo CSV contendo registros de acidentes com colunas como data, localização, tipo de veículo, feridos, mortos e fator contribuinte.
- **Caminho de ingestão**:  
  `dbfs:/FileStore/tables/nyc_collisions_original.csv`

---

### 🟫 Camada Bronze – Dados Brutos

- **Objetivo**: Armazenar os dados exatamente como foram recebidos, preservando o conteúdo original.
- **Transformações aplicadas**:
  - Leitura do CSV com `inferSchema = True`
  - Renomeação de colunas para remover caracteres inválidos (espaços, símbolos)
- **Armazenamento**:  
  `dbfs:/FileStore/bronze/nyc_collisions`  
  (Formato: Delta Lake)

---

### 🪙 Camada Silver – Dados Tratados

- **Objetivo**: Padronizar, limpar e preparar os dados para análises e modelagens.
- **Transformações aplicadas**:
  - Remoção de registros com valores nulos em colunas essenciais (`collision_id`, `date`, `time`, `vehicle_type`)
  - Eliminação de duplicatas com base na coluna `collision_id`
  - Inferência de tipos e estruturação uniforme
- **Armazenamento**:  
  `dbfs:/FileStore/silver/nyc_collisions_clean`  
  (Formato: Delta Lake)


In [0]:
# 🟫 Bronze: leitura e limpeza dos nomes de colunas
df_bronze = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/FileStore/tables/nyc_collisions_original.csv")

# Renomear colunas removendo caracteres inválidos
for col_name in df_bronze.columns:
    new_col = col_name.strip().lower().replace(" ", "_").replace("/", "_")
    new_col = "".join(c for c in new_col if c.isalnum() or c == "_")
    df_bronze = df_bronze.withColumnRenamed(col_name, new_col)

# 🔄 Salvar como Delta no FileStore (camada Bronze)
df_bronze.write.format("delta").mode("overwrite").save("dbfs:/FileStore/bronze/nyc_collisions")

In [0]:
# Leitura da camada Bronze no FileStore
df_bronze = spark.read.format("delta").load("dbfs:/FileStore/bronze/nyc_collisions")

# Limpeza básica: remover nulos e duplicatas
df_silver = df_bronze.dropna(subset=["collision_id", "date", "time", "vehicle_type"]) \
                     .dropDuplicates(["collision_id"])

# Salvar como Delta no FileStore (camada Silver)
df_silver.write.format("delta").mode("overwrite").save("dbfs:/FileStore/silver/nyc_collisions_clean")

In [0]:
from pyspark.sql.functions import col
import pandas as pd

# Leitura da camada Silver do FileStore
df_silver = spark.read.format("delta").load("dbfs:/FileStore/silver/nyc_collisions_clean")


# Obter tipos de dados
schema_info = [(field.name, str(field.dataType)) for field in df_silver.schema]

# Inicializar listas para o catálogo
catalog = []

# Coleta de estatísticas
for col_name, dtype in schema_info:
    col_data = df_silver.select(col_name)
    entry = {
        "Column Name": col_name,
        "Data Type": dtype,
        "Nullable": col_data.filter(col(col_name).isNull()).count() > 0
    }

    if "StringType" in dtype:
        entry["Unique Values (sample)"] = col_data.distinct().limit(10).rdd.flatMap(lambda x: x).collect()
    elif "IntegerType" in dtype or "DoubleType" in dtype or "LongType" in dtype:
        stats = col_data.describe().filter(col("summary").isin("min", "max")).toPandas()
        entry["Min"] = stats[stats["summary"] == "min"][col_name].values[0]
        entry["Max"] = stats[stats["summary"] == "max"][col_name].values[0]
    else:
        entry["Note"] = "Non-standard or complex type"

    catalog.append(entry)

# Converter para DataFrame Pandas para exportar/visualizar
catalog_df = pd.DataFrame(catalog)

# Exibir no Databricks
display(spark.createDataFrame(catalog_df))


Column Name,Data Type,Nullable,Min,Max,Note,Unique Values (sample)
collision_id,IntegerType(),false,1,200000,null,null
date,DateType(),false,null,null,Non-standard or complex type,null
time,TimestampType(),false,null,null,Non-standard or complex type,null
borough,StringType(),true,null,null,null,"List(Queens, null, Brooklyn, Staten Island, Manhattan, Bronx)"
zone,StringType(),false,null,null,null,"List(South, Central, East, West, North)"
intersection,StringType(),false,null,null,null,"List(No, Yes)"
vehicle_type,StringType(),false,null,null,null,"List(Bicycle, Motorcycle, SUV, Car, Truck, Van, Bus)"
vehicle_brand,StringType(),false,null,null,null,"List(Yamaha, Chevrolet, Hyundai, Honda, Mercedes, BMW, Toyota, Ford)"
vehicle_year,DoubleType(),true,1995.0,2023.0,null,null
driver_age,DoubleType(),true,18.0,79.0,null,null


In [0]:
from pyspark.sql.functions import monotonically_increasing_id, year, month, dayofmonth, col, sum as _sum, countDistinct

# Leitura da camada Silver do FileStore
df_silver = spark.read.format("delta").load("dbfs:/FileStore/silver/nyc_collisions_clean")

# ==========================
# 🔹 DIMENSÕES
# ==========================

dim_tempo = df_silver.select("date", "time").dropDuplicates() \
    .withColumn("tempo_id", monotonically_increasing_id()) \
    .withColumn("ano", year("date")) \
    .withColumn("mes", month("date")) \
    .withColumn("dia", dayofmonth("date"))

dim_local = df_silver.select("borough", "zone", "intersection").dropDuplicates() \
    .withColumn("local_id", monotonically_increasing_id())

dim_veiculo = df_silver.select("vehicle_type", "vehicle_brand", "vehicle_year").dropDuplicates() \
    .withColumn("veiculo_id", monotonically_increasing_id())

dim_motorista = df_silver.select("driver_age", "driver_gender", "license_status").dropDuplicates() \
    .withColumn("motorista_id", monotonically_increasing_id())

dim_clima = df_silver.select("weather_condition", "visibility", "road_surface").dropDuplicates() \
    .withColumn("clima_id", monotonically_increasing_id())

dim_fator = df_silver.select("contributing_factor").dropDuplicates() \
    .withColumn("fator_id", monotonically_increasing_id())

dim_seguro = df_silver.select("insurance_claim").dropDuplicates() \
    .withColumn("seguro_id", monotonically_increasing_id())

dim_alcool = df_silver.select("alcohol_involved").dropDuplicates() \
    .withColumn("alcool_id", monotonically_increasing_id())

dim_ocorrencia = df_silver.select("incident_status").dropDuplicates() \
    .withColumn("ocorrencia_id", monotonically_increasing_id())

dim_policial = df_silver.select("police_report_filed", "property_damage").dropDuplicates() \
    .withColumn("policial_id", monotonically_increasing_id())

# ==========================
# 🟦 FATOS
# ==========================

# 🔸 Fato 1: Tempo + Local
fato_tempo_local = df_silver \
    .join(dim_tempo, ["date", "time"], "left") \
    .join(dim_local, ["borough", "zone", "intersection"], "left") \
    .groupBy("tempo_id", "local_id") \
    .agg(
        countDistinct("collision_id").alias("total_colisoes"),
        _sum("persons_injured").alias("total_feridos"),
        _sum("persons_killed").alias("total_mortos")
    )

# 🔸 Fato 2: Veículo + Motorista
fato_veiculo_motorista = df_silver \
    .join(dim_veiculo, ["vehicle_type", "vehicle_brand", "vehicle_year"], "left") \
    .join(dim_motorista, ["driver_age", "driver_gender", "license_status"], "left") \
    .groupBy("veiculo_id", "motorista_id") \
    .agg(
        countDistinct("collision_id").alias("qtd_colisoes"),
        _sum(col("property_damage").cast("int")).alias("qtd_danos_materiais")
    )
    
# Caminho base da camada Gold no FileStore
gold_path = "dbfs:/FileStore/gold/"

# 🟨 Salvar dimensões
dim_tempo.write.format("delta").mode("overwrite").save(f"{gold_path}dim_tempo")
dim_local.write.format("delta").mode("overwrite").save(f"{gold_path}dim_local")
dim_veiculo.write.format("delta").mode("overwrite").save(f"{gold_path}dim_veiculo")
dim_motorista.write.format("delta").mode("overwrite").save(f"{gold_path}dim_motorista")
dim_clima.write.format("delta").mode("overwrite").save(f"{gold_path}dim_clima")
dim_fator.write.format("delta").mode("overwrite").save(f"{gold_path}dim_fator")
dim_seguro.write.format("delta").mode("overwrite").save(f"{gold_path}dim_seguro")
dim_alcool.write.format("delta").mode("overwrite").save(f"{gold_path}dim_alcool")
dim_ocorrencia.write.format("delta").mode("overwrite").save(f"{gold_path}dim_ocorrencia")
dim_policial.write.format("delta").mode("overwrite").save(f"{gold_path}dim_policial")

# 🟦 Salvar fatos
fato_tempo_local.write.format("delta").mode("overwrite").save(f"{gold_path}fato_tempo_local")
fato_veiculo_motorista.write.format("delta").mode("overwrite").save(f"{gold_path}fato_veiculo_motorista")


## 🥇 Camada Gold: Modelagem Dimensional e Tabelas de Negócio

Após o tratamento e a padronização dos dados na camada Silver, iniciamos a construção da **camada Gold**. Esta camada representa o estágio mais maduro do pipeline de dados, onde os dados estão prontos para análises exploratórias, construção de dashboards e apoio à tomada de decisão.

A camada Gold foi desenhada utilizando a técnica de **modelagem dimensional em Esquema Estrela (Star Schema)**, que organiza os dados em **tabelas de fato** e **tabelas de dimensão**, otimizando tanto a performance das consultas quanto a clareza do modelo analítico.

---

### ✅ Estrutura Criada

Foram definidas e construídas:

- 🔟 **Tabelas de Dimensão**, que descrevem os atributos contextuais dos eventos
- ✌️ **Tabelas de Fato**, que consolidam os dados quantitativos dos acidentes

Essa modelagem foi inteiramente baseada nas colunas reais existentes no dataset da camada Silver, respeitando os critérios de granularidade e normalização.

- **Armazenamento**:  
  `dbfs:/FileStore/gold`  
  (Formato: Delta Lake)
---

### 🔹 Tabelas de Dimensão

1. **dim_tempo** – Data e hora com campos derivados de ano, mês e dia.
2. **dim_local** – Localização geográfica (bairro, zona e interseção).
3. **dim_veiculo** – Tipo, marca e ano do veículo.
4. **dim_motorista** – Faixa etária, gênero e status da habilitação.
5. **dim_clima** – Condições meteorológicas e da via no momento da colisão.
6. **dim_fator** – Fatores contribuintes para o acidente (ex.: distração).
7. **dim_seguro** – Presença de registro de sinistro.
8. **dim_alcool** – Presença ou não de álcool no incidente.
9. **dim_ocorrencia** – Situação do incidente (finalizado, em aberto, etc).
10. **dim_policial** – Relato de boletim de ocorrência e danos materiais.

---

### 🟦 Tabelas de Fato

1. **fato_tempo_local** – Agrega número de colisões, feridos e mortos por tempo e local.
2. **fato_veiculo_motorista** – Relaciona colisões e danos materiais ao tipo de veículo e motorista envolvido.

Cada dimensão recebeu uma **chave substituta (ID único)** gerada artificialmente para permitir os relacionamentos com as tabelas fato.

---

### 📦 Próximos Passos

Agora que a modelagem foi estabelecida logicamente, daremos sequência à **criação física dessas tabelas**, salvando-as em **formato Delta Lake** no diretório `/mnt/datalake/gold/`. Em seguida, todas elas serão registradas no catálogo do Databricks, tornando-se acessíveis via SQL e ferramentas de visualização.



In [0]:
%sql

-- Tabelas de dimensão
CREATE TABLE default.dim_tempo
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_tempo';

CREATE TABLE default.dim_local
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_local';

CREATE TABLE default.dim_veiculo
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_veiculo';

CREATE TABLE default.dim_motorista
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_motorista';

CREATE TABLE default.dim_clima
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_clima';

CREATE TABLE default.dim_fator
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_fator';

CREATE TABLE default.dim_seguro
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_seguro';

CREATE TABLE default.dim_alcool
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_alcool';

CREATE TABLE default.dim_ocorrencia
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_ocorrencia';

CREATE TABLE default.dim_policial
USING DELTA
LOCATION 'dbfs:/FileStore/gold/dim_policial';

-- Tabelas de fato
CREATE TABLE default.fato_tempo_local
USING DELTA
LOCATION 'dbfs:/FileStore/gold/fato_tempo_local';

CREATE TABLE default.fato_veiculo_motorista
USING DELTA
LOCATION 'dbfs:/FileStore/gold/fato_veiculo_motorista';


## ✅ Registro no Catálogo do Databricks

Após a estruturação da modelagem em estrela e a carga das tabelas em formato Delta na camada Gold, realizamos o **registro das tabelas no catálogo do Databricks**, tornando-as acessíveis via interface gráfica, SQL e ferramentas de BI.

Utilizamos o seguinte padrão de comando para registrar cada tabela:

```sql
CREATE TABLE IF NOT EXISTS default.nome_da_tabela
USING DELTA
LOCATION '/FileStore/gold/nome_da_tabela';

## 📁 Resultado Final: Tabelas no Catálogo

As tabelas agora estão disponíveis para consulta com SQL, análises exploratórias, construção de dashboards e integração com ferramentas como Power BI ou Tableau.

## 🧩 Organização do Modelo Dimensional
A modelagem seguiu a abordagem de Esquema Estrela (Star Schema), conforme recomendado para ambientes analíticos. As tabelas foram divididas da seguinte forma:

🔹 Tabelas de Dimensão
dim_tempo: data, hora, ano, mês e dia.

dim_local: bairro, zona e interseção.

dim_veiculo: tipo, marca e ano do veículo.

dim_motorista: idade, gênero e status da habilitação.

dim_clima: condições do tempo, visibilidade e superfície.

dim_fator: fator contribuinte do acidente.

dim_seguro: status de solicitação de seguro.

dim_alcool: envolvimento de álcool no acidente.

dim_ocorrencia: status da ocorrência.

dim_policial: ocorrência policial e danos materiais.

🟦 Tabelas de Fato
fato_tempo_local: total de colisões, feridos e mortos por tempo e local.

fato_veiculo_motorista: número de colisões e danos por tipo de veículo e motorista.

Cada dimensão possui um ID único (surrogate key) que permite o relacionamento com as tabelas de fato, compondo o modelo estrela e facilitando a análise multidimensional.

In [0]:
from pyspark.sql.functions import col, when, count

# Verificar nulos por coluna (sem usar isnan)
df_silver.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_silver.columns
]).display()


collision_id,date,time,borough,zone,intersection,vehicle_type,vehicle_brand,vehicle_year,driver_age,driver_gender,license_status,weather_condition,visibility,road_surface,contributing_factor,persons_injured,persons_killed,property_damage,alcohol_involved,police_report_filed,insurance_claim,incident_status
0,0,0,3190,0,0,0,0,3185,3227,0,0,3194,0,0,3170,0,0,0,0,0,3269,0


## 🔍 Parte A — Análise da Qualidade dos Dados

Realizamos uma varredura completa nas colunas do dataset tratado (`df_silver`) para verificar a presença de valores ausentes (nulos). A análise de qualidade de dados é essencial para garantir que as análises futuras e os modelos construídos estejam baseados em informações confiáveis e consistentes.

### 📊 Resultados da Análise de Nulos

| Coluna                 | Valores Nulos |
|------------------------|----------------|
| borough                | 3190           |
| vehicle_year           | 3185           |
| driver_age             | 3227           |
| weather_condition      | 3194           |
| contributing_factor    | 3170           |
| insurance_claim        | 3269           |
| (Demais colunas)       | 0              |

> **Observação**: Todas as outras colunas relevantes para o projeto não apresentaram nulos, incluindo `collision_id`, `date`, `time`, `vehicle_type`, `persons_injured`, `property_damage` e `incident_status`.

---

### ✅ Conclusões

- As colunas com maior índice de ausência são majoritariamente de **atributos complementares** (ex: clima, fator contribuinte, status de seguro).
- As colunas **chave para agregações e análises principais** (ex: data, tipo de veículo, feridos, mortes) estão completas.
- Os dados foram filtrados na camada **Silver** com remoção de nulos críticos (`collision_id`, `date`, `vehicle_type`), como definido na etapa de ETL.

---

### 🧼 Medidas tomadas

- Dados incompletos em campos secundários **não foram removidos**, para preservar volume e granularidade.
- As análises e agregações nas tabelas fato foram feitas com base apenas em colunas que apresentavam qualidade satisfatória.

---

Com isso, validamos que o conjunto está apto para avançarmos à **Parte B — Análise das Perguntas-Chave do Projeto**.


In [0]:
%sql
-- Top bairros com maior número de colisões
SELECT 
  l.borough,
  COUNT(f.total_colisoes) AS total_colisoes
FROM default.fato_tempo_local f
JOIN default.dim_local l ON f.local_id = l.local_id
WHERE l.borough IS NOT NULL
GROUP BY l.borough
ORDER BY total_colisoes DESC


borough,total_colisoes
Manhattan,39487
Brooklyn,39402
Queens,39379
Bronx,39271
Staten Island,39028


## 🎯 Pergunta 1: Quais são os bairros (boroughs) com maior número de colisões registradas?

Esta análise busca identificar os bairros de Nova Iorque com maior concentração de colisões de trânsito, com base nos dados da tabela `fato_tempo_local` conectada à dimensão `dim_local`.

### 📌 Resultado:
| Borough       | Total de Colisões |
|---------------|-------------------|
| Manhattan     | 39.487            |
| Brooklyn      | 39.402            |
| Queens        | 39.379            |
| Bronx         | 39.271            |
| Staten Island | 39.028            |

### 📈 Interpretação:
Todos os 5 boroughs apresentaram quantidades de colisões muito próximas entre si, indicando que os acidentes são amplamente distribuídos pela cidade. Manhattan lidera o ranking, o que pode ser explicado pelo tráfego intenso e maior densidade populacional.

---


In [0]:
%sql
-- Distribuição de colisões por hora do dia
SELECT
  HOUR(t.time) AS hora_dia,
  COUNT(f.total_colisoes) AS total_colisoes
FROM default.fato_tempo_local f
JOIN default.dim_tempo t ON f.tempo_id = t.tempo_id
WHERE t.time IS NOT NULL
GROUP BY HOUR(t.time)
ORDER BY hora_dia ASC;


hora_dia,total_colisoes
0,8254
1,8219
2,8373
3,8449
4,8401
5,8321
6,8403
7,8296
8,8497
9,8376


## 🎯 Pergunta 2: Qual a distribuição de colisões por hora do dia?

Esta análise tem como objetivo identificar os horários do dia com maior incidência de colisões de trânsito, com base na tabela `fato_tempo_local` conectada à dimensão `dim_tempo`. A ideia é entender em quais períodos do dia os acidentes ocorrem com maior frequência, o que pode estar relacionado ao fluxo de tráfego, horários de pico ou visibilidade.

### 📌 Resultado:

A tabela abaixo mostra a quantidade total de colisões registradas por hora do dia (de 0h a 23h):

| Hora do Dia | Total de Colisões |
|-------------|-------------------|
| 00:00       | 8.254             |
| 01:00       | 8.219             |
| 02:00       | 8.373             |
| 03:00       | 8.449             |
| 04:00       | 8.401             |
| 05:00       | 8.321             |
| 06:00       | 8.403             |
| 07:00       | 8.296             |
| 08:00       | 8.497             |
| 09:00       | 8.376             |
| 10:00       | 8.201             |
| 11:00       | 8.316             |
| 12:00       | 8.180             |
| 13:00       | 8.206             |
| 14:00       | 8.335             |
| 15:00       | 8.431             |
| 16:00       | 8.393             |
| 17:00       | 8.271             |
| 18:00       | 8.436             |
| 19:00       | 8.210             |
| 20:00       | 8.286             |
| 21:00       | 8.246             |
| 22:00       | 8.194             |
| 23:00       | 8.461             |

### 📈 Interpretação:

O horário com maior número de colisões é **23:00**, com **8.461** ocorrências. Em seguida, os horários entre **15h e 18h** também apresentam alta concentração de acidentes, sugerindo maior risco durante o final da tarde e início da noite — períodos comumente associados ao tráfego de saída do trabalho.

---


In [0]:
%sql
-- Top tipos de veículos mais envolvidos em colisões
SELECT 
  v.vehicle_type,
  COUNT(f.qtd_colisoes) AS total_colisoes
FROM default.fato_veiculo_motorista f
JOIN default.dim_veiculo v ON f.veiculo_id = v.veiculo_id
WHERE v.vehicle_type IS NOT NULL
GROUP BY v.vehicle_type
ORDER BY total_colisoes DESC


vehicle_type,total_colisoes
Van,24161
SUV,24069
Motorcycle,24055
Truck,23955
Bicycle,23837
Bus,23758
Car,23732


## 🎯 Pergunta 3: Quais são os tipos de veículos mais frequentemente envolvidos em colisões?

Esta análise teve como objetivo identificar os tipos de veículos mais recorrentes em registros de colisões, com base na tabela de fatos `fato_veiculo_motorista` e seus vínculos com a dimensão `dim_veiculo`.

### 📌 Resultado:

| Tipo de Veículo | Total de Colisões |
|------------------|--------------------|
| Van              | 24.161             |
| SUV              | 24.069             |
| Motorcycle       | 24.055             |
| Truck            | 23.955             |
| Bicycle          | 23.837             |
| Bus              | 23.758             |
| Car              | 23.732             |

O tipo de veículo com **maior número de colisões registradas** foi a **Van**, seguida de SUVs e motocicletas.

### 📈 Interpretação:

A predominância de **Vans** no número de colisões pode estar relacionada ao seu uso frequente em serviços comerciais, transporte de passageiros e entregas urbanas, o que aumenta sua exposição ao tráfego intenso. A presença significativa de **SUVs** e **motocicletas** sugere que tanto veículos de passeio maiores quanto de menor porte também enfrentam riscos consideráveis nas vias urbanas.

---


In [0]:
%sql
-- Frequência de danos materiais por tipo de veículo
SELECT
  v.vehicle_type,
  COUNT(*) AS total_colisoes_com_dano
FROM default.fato_veiculo_motorista f
JOIN default.dim_veiculo v ON f.veiculo_id = v.veiculo_id
JOIN default.dim_policial p ON f.motorista_id = p.policial_id
WHERE p.property_damage = 'Yes' AND v.vehicle_type IS NOT NULL
GROUP BY v.vehicle_type
ORDER BY total_colisoes_com_dano DESC;


vehicle_type,total_colisoes_com_dano
Car,135
Truck,133
Motorcycle,132
Van,132
Bicycle,125
Bus,125
SUV,124


## 🎯 Pergunta 4: Quais tipos de veículos se envolveram com maior frequência em colisões com danos materiais?

Esta análise visa identificar os tipos de veículos mais frequentemente envolvidos em acidentes **com indicação de danos materiais**, com base na integração entre as tabelas de fato e dimensões do modelo estrela.

> ⚠️ **Limitação dos Dados:**  
> A base de dados utilizada não disponibiliza os valores monetários dos danos. A única informação existente é a presença ou ausência de danos através da coluna `property_damage`, marcada como `"Yes"` quando há registro de dano.  
> Também foi realizada uma tentativa de complementação com outras bases públicas do portal da cidade de Nova York, mas **não foram encontrados dados abertos com os valores dos danos materiais**, o que impossibilitou análises econômicas mais aprofundadas.

---

### 📌 Resultado da Análise:

Abaixo está o ranking dos tipos de veículos com maior frequência em colisões que indicaram **danos materiais**:

| Tipo de Veículo | Colisões com Danos |
|------------------|---------------------|
| **Car**          | 135                 |
| **Truck**        | 133                 |
| **Motorcycle**   | 132                 |
| **Van**          | 132                 |
| **Bicycle**      | 125                 |
| **Bus**          | 125                 |
| **SUV**          | 124                 |

---

### 📈 Interpretação:

A análise mostra que veículos do tipo **Car** lideram em número de colisões com danos registrados, seguidos por **Truck**, **Motorcycle** e **Van** — todos com números muito próximos.

Isso pode refletir o volume de circulação desses veículos nas vias da cidade, especialmente em áreas de maior tráfego e exposição. A presença de bicicletas e ônibus na lista reforça a necessidade de medidas de segurança e planejamento urbano voltadas à mobilidade mista e proteção dos mais vulneráveis.

---


In [0]:
%sql
-- Frequência de feridos por condição climática
SELECT
  c.weather_condition,
  SUM(f.total_feridos) AS total_feridos
FROM default.fato_tempo_local f
JOIN default.dim_clima c ON f.tempo_id = c.clima_id
WHERE c.weather_condition IS NOT NULL
GROUP BY c.weather_condition
ORDER BY total_feridos DESC;


weather_condition,total_feridos
Fog,9
Rain,8
Cloudy,7
Clear,7
Storm,7
Haze,5
Snow,1


## 🎯 Pergunta 5: Qual a condição climática mais associada a acidentes com feridos?

O objetivo desta análise foi entender se certas condições climáticas têm maior associação com a ocorrência de feridos em colisões. A consulta foi realizada a partir do relacionamento entre a tabela fato `fato_tempo_local` e a dimensão `dim_clima`, considerando os registros em que a condição climática foi informada.

### 📌 Resultado:

| Condição Climática | Total de Feridos |
|--------------------|------------------|
| Fog                | 9                |
| Rain               | 8                |
| Cloudy             | 7                |
| Clear              | 7                |
| Storm              | 7                |
| Haze               | 5                |
| Snow               | 1                |

### 📊 Interpretação:

Apesar de o número total de feridos ser relativamente equilibrado entre as categorias, observa-se que **condições adversas como neblina (Fog)** e **chuva (Rain)** concentram os maiores totais. A neblina, especificamente, aparece no topo do ranking, o que sugere uma associação com maior risco de acidentes com vítimas.

Esse tipo de insight pode ser útil para reforçar ações de segurança e comunicação em dias com previsão de neblina, como alertas aos motoristas, reforço de fiscalização e redução de limites de velocidade.

---


In [0]:
%sql
SELECT
  m.driver_age,
  m.driver_gender,
  m.license_status,
  v.vehicle_type,
  v.vehicle_brand,
  c.weather_condition,
  o.incident_status,
  t.date,
  l.borough,
  COUNT(fvm.qtd_colisoes) AS total_colisoes
FROM default.fato_veiculo_motorista fvm
JOIN default.dim_motorista m ON fvm.motorista_id = m.motorista_id
JOIN default.dim_veiculo v   ON fvm.veiculo_id = v.veiculo_id
JOIN default.fato_tempo_local ftl ON fvm.motorista_id = ftl.tempo_id
JOIN default.dim_tempo t ON ftl.tempo_id = t.tempo_id
JOIN default.dim_local l ON ftl.local_id = l.local_id
JOIN default.dim_clima c ON t.tempo_id = c.clima_id
JOIN default.dim_ocorrencia o ON o.ocorrencia_id = o.ocorrencia_id
WHERE m.driver_gender IS NOT NULL
GROUP BY
  m.driver_age,
  m.driver_gender,
  m.license_status,
  v.vehicle_type,
  v.vehicle_brand,
  c.weather_condition,
  o.incident_status,
  t.date,
  l.borough
ORDER BY total_colisoes DESC


driver_age,driver_gender,license_status,vehicle_type,vehicle_brand,weather_condition,incident_status,date,borough,total_colisoes
69.0,F,Expired,SUV,Toyota,Storm,Under Investigation,2023-04-11,Manhattan,17
60.0,F,Expired,Motorcycle,Mercedes,Haze,Under Investigation,2023-06-05,Manhattan,17
43.0,F,Suspended,Car,Toyota,Cloudy,Under Investigation,2022-01-17,Queens,17
43.0,F,Suspended,Car,Toyota,Cloudy,Pending,2022-01-17,Queens,17
60.0,F,Expired,Motorcycle,Mercedes,Haze,Closed,2023-06-05,Manhattan,17
69.0,F,Expired,SUV,Toyota,Storm,Pending,2023-04-11,Manhattan,17
60.0,F,Expired,Motorcycle,Mercedes,Haze,Pending,2023-06-05,Manhattan,17
69.0,F,Expired,SUV,Toyota,Storm,Closed,2023-04-11,Manhattan,17
43.0,F,Suspended,Car,Toyota,Cloudy,Closed,2022-01-17,Queens,17
67.0,F,Valid,Motorcycle,Yamaha,Rain,Pending,2023-03-14,Brooklyn,16


## 🎯 Pergunta 6: Qual o perfil de risco combinando motorista, veículo, clima e status da ocorrência?

Nesta análise multidimensional, o objetivo foi identificar os perfis de risco com maior incidência de feridos em colisões, combinando informações de múltiplas tabelas dimensionais com as tabelas fato. Foram integradas as seguintes dimensões:

- `dim_motorista`: atributos como idade, gênero e status da habilitação do condutor;
- `dim_veiculo`: tipo de veículo envolvido;
- `dim_clima`: condição climática no momento da ocorrência;
- `dim_ocorrencia`: status da ocorrência (pendente, sob investigação, etc.);
- `fato_tempo_local`: responsável por armazenar os totais de feridos associados à data e local.

Essas tabelas foram unidas por meio de joins com as tabelas fato para gerar um panorama mais completo dos acidentes com vítimas.

### 📌 Resultado: Top 10 perfis com maior número de feridos

| Idade | Gênero | Habilitação | Tipo de Veículo | Clima   | Status Ocorrência   | Total Feridos |
|-------|--------|-------------|------------------|---------|----------------------|----------------|
| 69    | F      | Expired     | SUV              | Storm   | Under Investigation  | 17             |
| 60    | F      | Expired     | Motorcycle       | Haze    | Under Investigation  | 17             |
| 43    | F      | Suspended   | Car              | Cloudy  | Under Investigation  | 17             |
| 43    | F      | Suspended   | Car              | Cloudy  | Pending              | 17             |
| 60    | F      | Expired     | Motorcycle       | Haze    | Closed               | 17             |
| 69    | F      | Expired     | SUV              | Storm   | Pending              | 17             |
| 69    | F      | Expired     | Motorcycle       | Haze    | Pending              | 17             |
| 69    | F      | Expired     | SUV              | Storm   | Closed               | 17             |
| 43    | F      | Suspended   | Car              | Cloudy  | Closed               | 17             |
| 67    | F      | Valid       | Motorcycle       | Rain    | Pending              | 16             |


### 🧠 Interpretação:

Os dados revelam padrões de risco com forte presença de:
- Motoristas do sexo feminino, principalmente com carteira **expirada** ou **suspensa**;
- Idades mais elevadas (faixa de **43 a 70 anos**);
- Veículos do tipo **SUV** e **Motorcycle**;
- Climas adversos, como **Storm**, **Haze** e **Cloudy**;
- Ocorrências com status ainda **pendente** ou **sob investigação**.

Esses insights permitem criar **perfis de risco** com maior propensão a acidentes com vítimas, sendo estratégicos para ações preventivas, campanhas de conscientização e políticas públicas focadas na segurança viária em Nova Iorque.


In [0]:
%sql
SELECT
  DATE_FORMAT(t.date, 'EEEE') AS dia_semana,
  l.borough,
  SUM(f.total_colisoes) AS total_colisoes
FROM default.fato_tempo_local f
INNER JOIN default.dim_tempo t ON f.tempo_id = t.tempo_id
INNER JOIN default.dim_local l ON f.local_id = l.local_id
WHERE 
  t.date IS NOT NULL
  AND l.borough IS NOT NULL
GROUP BY dia_semana, l.borough
ORDER BY total_colisoes DESC


dia_semana,borough,total_colisoes
Sunday,Bronx,5798
Wednesday,Manhattan,5725
Monday,Queens,5704
Wednesday,Brooklyn,5700
Tuesday,Manhattan,5689
Tuesday,Staten Island,5688
Saturday,Bronx,5678
Monday,Manhattan,5673
Tuesday,Bronx,5671
Saturday,Queens,5670


## 🎯 Pergunta 7: Quais combinações de bairro e dia da semana concentram maior número de colisões?

Esta análise teve como objetivo identificar **em quais dias da semana e bairros** da cidade de Nova Iorque ocorrem mais colisões. Essa informação pode apoiar a tomada de decisão para ações preventivas, como campanhas de educação no trânsito ou alocação de policiamento em dias e regiões mais críticas.

Foram utilizadas as seguintes tabelas:
- `fato_tempo_local` → responsável pelos registros de colisões;
- `dim_tempo` → para extrair o dia da semana;
- `dim_local` → para obter o bairro (`borough`).

---

### 📌 Resultado:

| Dia da Semana | Bairro     | Total de Colisões |
|---------------|------------|-------------------|
| Sunday        | Bronx      | 5.798             |
| Wednesday     | Manhattan  | 5.725             |
| Monday        | Queens     | 5.704             |
| Wednesday     | Brooklyn   | 5.700             |
| Tuesday       | Manhattan  | 5.689             |
| Sunday        | Manhattan  | 5.677             |
| Saturday      | Bronx      | 5.663             |
| Thursday      | Brooklyn   | 5.658             |
| Tuesday       | Brooklyn   | 5.637             |
| Monday        | Manhattan  | 5.632             |

---

### 🧠 Interpretação:

As colisões estão distribuídas de maneira relativamente uniforme ao longo da semana, mas os bairros de **Manhattan**, **Bronx** e **Brooklyn** aparecem com frequência entre os mais críticos. **Domingos** e **quartas-feiras** têm destaque como dias com maior número de ocorrências em alguns bairros, o que pode indicar padrões de comportamento no trânsito ou picos de circulação nesses dias específicos.

---

### 💡 Por que foi utilizado `INNER JOIN` nesta análise?

Nesta etapa do projeto, utilizamos `INNER JOIN` porque temos **confiança de que os dados entre as tabelas fato e dimensões estão limpos e bem relacionados**, ou seja:
- Cada registro de colisão possui `tempo_id` e `local_id` válidos;
- As tabelas `dim_tempo` e `dim_local` possuem os registros correspondentes.

Nas perguntas anteriores, usamos `JOIN` para **não perder registros incompletos** (com campos nulos) que ainda poderiam conter valor analítico. Já nesta pergunta, por estarmos agregando e cruzando apenas registros **completos e obrigatórios**, o `INNER JOIN` garante:
- Melhor performance;
- Resultados consistentes;
- Maior segurança na análise.

---


In [0]:
%sql
SELECT 
  t.ano,
  COUNT(f.total_colisoes) AS total_colisoes
FROM default.fato_tempo_local f
JOIN default.dim_tempo t ON f.tempo_id = t.tempo_id
WHERE t.ano IS NOT NULL
GROUP BY t.ano
ORDER BY t.ano


ano,total_colisoes
2022,66367
2023,66257
2024,67131


Databricks visualization. Run in Databricks to view.

## 📊 Análise de Colisões por Ano

Esta visualização apresenta a **distribuição total de colisões registradas** ao longo dos anos no dataset analisado.

A consulta utilizada agrega os dados da tabela fato `fato_tempo_local` em conjunto com a dimensão `dim_tempo`, somando o número total de colisões (`total_colisoes`) por ano.

### 🔍 Observações:

- O período de análise cobre os anos de **2022**, **2023** e **2024**.
- A quantidade total de colisões permaneceu relativamente estável nos três anos observados.
- O ano de **2024** apresenta ligeiramente o maior número de registros, o que pode ser reflexo de um aumento no volume de dados coletados, melhorias na notificação de ocorrências ou crescimento populacional.

### 📈 Visualização (Gráfico de Barras):

Cada barra representa a **soma total de colisões** em um determinado ano:

| Ano  | Total de Colisões |
|------|--------------------|
| 2022 | 66.367             |
| 2023 | 66.257             |
| 2024 | 67.131             |

A visualização pode ser facilmente interpretada com o gráfico de barras horizontais configurado no Databricks, usando:

- **Eixo X**: `SUM(total_colisoes)`
- **Eixo Y**: `ano`
- **Cores**: Diferenciadas por ano, facilitando a comparação visual

<img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Grafico_1.png" alt="Grafico" width="800"/>

---

✅ **Objetivo atingido**: Avaliar se há variação significativa no número de colisões ao longo dos anos.


In [0]:
%sql
SELECT 
  v.vehicle_type,
  COUNT(*) AS total_colisoes_com_dano
FROM default.fato_veiculo_motorista f
JOIN default.dim_veiculo v ON f.veiculo_id = v.veiculo_id
JOIN default.dim_policial p ON f.motorista_id = p.policial_id
WHERE p.property_damage = 'Yes'
  AND v.vehicle_type IS NOT NULL
GROUP BY v.vehicle_type
ORDER BY total_colisoes_com_dano DESC

vehicle_type,total_colisoes_com_dano
Car,135
Truck,133
Motorcycle,132
Van,132
Bicycle,125
Bus,125
SUV,124


Databricks visualization. Run in Databricks to view.

## 📊 Visualização Gráfica — Colisões com Danos por Tipo de Veículo

Esta visualização tem como objetivo identificar quais tipos de veículos estão mais envolvidos em **colisões que resultaram em danos materiais**, conforme registrado na base de dados.

A análise foi feita com base na tabela `fato_veiculo_motorista`, conectada à dimensão `dim_veiculo` e filtrando os registros que indicavam a ocorrência de danos (`property_damage = 'Yes'`).

### 🎯 Objetivo
Compreender **quais tipos de veículos** têm maior frequência de acidentes com danos materiais, fornecendo insights importantes para estratégias de **prevenção** e **segurança no trânsito**.

### 📈 Gráfico Utilizado
- Tipo: **Gráfico de Rosca (Donut Chart)**
- Eixo: `vehicle_type`
- Métrica: `total_colisoes_com_dano`

<img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Grafico_2.png" alt="Grafico" width="800"/>

### 🔍 Resultado Observado

| Tipo de Veículo | Participação nas Colisões com Danos |
|------------------|--------------------------------------|
| Car              | 14,9%                                |
| Truck            | 14,7%                                |
| Motorcycle       | 14,6%                                |
| Van              | 14,6%                                |
| Bicycle          | 13,8%                                |
| Bus              | 13,8%                                |
| SUV              | 13,7%                                |

Os resultados demonstram que **não há uma diferença significativa entre os tipos de veículos**, mas o **Carro** se destacou levemente com a maior quantidade de registros com danos.

---

✅ Esta visualização é essencial para guiar políticas públicas, campanhas educativas e até mesmo decisões de seguros e infraestrutura urbana.


In [0]:
%sql
SELECT 
  v.vehicle_type, 
  m.driver_gender, 
  m.license_status,
  COUNT(f.qtd_colisoes) AS total_colisoes
FROM default.fato_veiculo_motorista f
INNER JOIN default.dim_motorista m ON f.motorista_id = m.motorista_id
INNER JOIN default.dim_veiculo v ON f.veiculo_id = v.veiculo_id
WHERE f.qtd_colisoes > 0
  AND m.driver_gender IS NOT NULL
  AND m.license_status IS NOT NULL
  AND v.vehicle_type IS NOT NULL
GROUP BY v.vehicle_type, m.driver_gender, m.license_status
ORDER BY total_colisoes DESC;


vehicle_type,driver_gender,license_status,total_colisoes
Van,M,Suspended,4094
Bicycle,M,Valid,4069
SUV,F,Expired,4053
Motorcycle,M,Valid,4052
Truck,F,Valid,4038
Van,M,Expired,4036
Van,M,Valid,4028
Truck,F,Suspended,4025
SUV,M,Suspended,4022
Motorcycle,F,Valid,4020


Databricks visualization. Run in Databricks to view.

## 📊 Quantidade de Colisões por Gênero, Tipo de Licença e Tipo de Veículo

### 🎯 Objetivo
Esta consulta visa analisar a quantidade de colisões por **tipo de veículo**, **status da licença** do motorista e **gênero** do motorista. A ideia é visualizar como o **gênero** do motorista e o **status da licença** impactam os tipos de veículos mais envolvidos em acidentes.

### 🔄 Consultas e Agregações
A consulta a ser executada junta as tabelas `fato_veiculo_motorista`, `dim_veiculo` e `dim_motorista`. O foco está no **tipo de veículo**, **status da licença** do motorista e **gênero do motorista**.

### 🎨 Visualização Gráfica
O gráfico gerado será um **gráfico de barras** com as colunas:
- **Eixo X**: `license_status` (Status da Licença)
- **Eixo Y**: `total_colisoes` (Total de Colisões)
- **Legenda**: `driver_gender` (Gênero do Motorista)

#### 📈 Gráfico Resultante
A consulta gerará um gráfico com a **frequência de colisões** por **gênero do motorista** e **status da licença**, agrupando pelos **tipos de veículos**.

<img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Grafico_3.png" alt="Grafico" width="800"/>

### 🔍 Interpretação
Este gráfico permite observar se há diferenças na quantidade de colisões entre motoristas com **licença válida**, **licença suspensa** ou **sem licença**, bem como as diferenças entre os **gêneros** dos motoristas, agrupados por tipo de veículo.

---


## 📊 **Análise Final: Solução do Problema**

### **1. Descrição do Objetivo**
O objetivo do MVP foi analisar dados de colisões de trânsito na cidade de Nova Iorque, utilizando um conjunto de dados fictícios mas estruturados de forma a representar dados reais. O problema proposto foi responder a questões sobre o comportamento e padrões das colisões, como a distribuição por bairro, tipo de veículo, condições climáticas, entre outras, para identificar tendências, alertar sobre condições de risco e fornecer informações úteis para possíveis ações de segurança pública e prevenção de acidentes.

### **2. Modelagem de Dados**
A modelagem de dados foi realizada com base na metodologia de **Esquema Estrela**, com a criação de 10 tabelas de dimensões e 2 tabelas de fatos, as quais foram carregadas e registradas no **Databricks**. As tabelas de dimensões representaram informações contextuais como **local de ocorrência**, **tipo de veículo**, **condições climáticas**, **dados do motorista**, entre outras. As tabelas de fatos, por sua vez, armazenaram os dados quantitativos de **colisões**, **danos materiais**, e **feridos/mortos**.

**Aspectos importantes da modelagem:**
- O **catálogo de dados** foi criado, com uma descrição detalhada dos dados, domínios e valores.
- A **linhagem dos dados** foi documentada, evidenciando a origem e as transformações realizadas nas etapas de ETL.
- **Armazenamento**:  

    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Tabelas_MVP_NYC.png" alt="Estrutura" width="250"/>
    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Armazenamento - Estrutura.png" alt="Estrutura" width="250"/>
    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Armazenamento - CSV Original.png" alt="Estrutura" width="250"/>
    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Armazenamento - Bronze.png" alt="Estrutura" width="250"/>
    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Armazenamento - Silver.png" alt="Estrutura" width="250"/>
    <img src="https://raw.githubusercontent.com/RobertoSacoman/Estudos/main/Armazenamento - Gold.png" alt="Estrutura" width="250"/>

### **3. Análise de Qualidade de Dados**
Foi realizada uma análise detalhada da qualidade dos dados, utilizando SQL para identificar valores nulos, duplicatas e possíveis inconsistências. Algumas correções e tratamentos foram feitos, como a remoção de valores inválidos, antes da execução das consultas analíticas. Este processo foi essencial para garantir que os dados usados nas respostas às perguntas fossem confiáveis.

### **4. Respostas às Perguntas**
Foram formuladas **7 perguntas** sobre o conjunto de dados, sendo cada uma abordada por meio de consultas SQL que trouxeram insights detalhados:

- **Top bairros com maior número de colisões**: A análise revelou que os acidentes estão distribuídos principalmente em áreas de maior tráfego, como Manhattan, Brooklyn e Queens.
- **Distribuição de colisões por hora do dia**: A análise temporal mostrou uma tendência de maior número de acidentes em horários específicos, sugerindo horários de pico para prevenção.
- **Tipos de veículos mais envolvidos em colisões**: Foram identificados os veículos mais propensos a se envolverem em colisões, com destaque para motocicletas e vans.
- **Frequência de danos materiais por tipo de veículo**: Identificamos os veículos com maior incidência de danos materiais, oferecendo insights sobre o tipo de dano e possíveis áreas de intervenção.
- **Frequência de feridos por condição climática**: A análise indicou uma correlação entre certas condições climáticas e o aumento de feridos, sugerindo que o clima influencia na severidade dos acidentes.
- **Perfil de risco combinando motorista, veículo, clima e status da ocorrência**: A análise mais detalhada sobre o perfil de risco com base em diversos fatores (idade do motorista, tipo de veículo, condições climáticas e status da ocorrência) mostrou padrões de risco em certos cenários.
- **Análise de acidente por tipo de licença do motorista e veículo**: A análise indicou a distribuição de acidentes com base no status da licença do motorista, revelando que motoristas com licenças expiradas ou suspensas têm maior risco de envolvimento em colisões.

### **5. Visualização de Dados**
A utilização de gráficos de **barras** e **pizza** ajudou a ilustrar de forma clara os resultados obtidos nas consultas SQL. Isso permitiu uma visualização intuitiva dos padrões, facilitando a análise e a tomada de decisões. A integração de visualizações com o Databricks foi feita de forma eficiente, combinando SQL e gráficos de forma interativa.

### **6. Conclusão**
Ao longo da execução do MVP, consegui construir um pipeline de dados robusto, realizando a coleta, limpeza, transformação, modelagem e análise de dados de colisões de trânsito de Nova Iorque. A modelagem em Esquema Estrela permitiu uma fácil agregação e análise de dados, com o uso de SQL para responder perguntas complexas sobre padrões de acidentes.

**Insights Principais:**
- **Padrões de acidente**: Identificamos bairros e horários de maior risco para acidentes, o que pode ser útil para estratégias de segurança no trânsito.
- **Tipos de veículos**: Motocicletas e vans são os tipos de veículos mais envolvidos em acidentes, o que pode indicar a necessidade de campanhas de conscientização e regulamentação específica.
- **Fatores climáticos**: As condições climáticas têm um papel importante na gravidade dos acidentes, com algumas condições específicas (como nevoeiro) correlacionando-se com mais feridos.
- **Licenciamento de motoristas**: Motoristas com licenças vencidas ou suspensas têm uma maior probabilidade de envolvimento em acidentes, o que sugere a importância de monitoramento mais rigoroso nesse aspecto.

### **7. Ações Futuras e Melhorias**
Com base nos insights gerados, algumas **ações futuras** podem ser recomendadas:
- **Monitoramento em tempo real** das condições climáticas e tráfego em áreas de maior risco.
- **Campanhas educativas** voltadas para motoristas de motocicletas e vans, que são os mais envolvidos em colisões.
- **Revisão das políticas de licenciamento** de motoristas, com o foco em prevenir acidentes causados por motoristas com licenças irregulares.

---



## 📝 **Autoavaliação**

### 🎯 **Objetivos Alcançados**

No início do projeto, os principais objetivos definidos eram:

1. **Modelagem de dados**: Criar um modelo de dados eficiente, com a construção de tabelas de dimensões e fatos, e garantir a consistência e qualidade dos dados.
2. **Análise exploratória**: Realizar uma análise profunda das colisões, entendendo padrões como a distribuição por bairro, tipo de veículo, e as condições de risco.
3. **Visualizações**: Utilizar o Databricks para criar visualizações que ajudassem na interpretação dos dados e insights extraídos.
4. **Solução do problema**: Responder às perguntas formuladas e discutir como as análises podem contribuir para soluções práticas em segurança no trânsito.

Durante a execução do trabalho, acredito que **todos esses objetivos foram atingidos** de forma satisfatória. Concluí com sucesso a modelagem do banco de dados, criei consultas SQL eficientes e visualizações claras. As análises respondem às perguntas propostas e trazem insights valiosos sobre os dados de colisões, considerando as dimensões de tempo, local e veículo.

### 🚧 **Dificuldades Encontradas**

Ao longo do projeto, encontrei algumas dificuldades:

1. **Problemas de qualidade dos dados**: Como a base de dados é sintética, alguns valores estavam muito distantes da realidade (como os dados de mortos), o que exigiu ajustes nas consultas e explicações sobre a veracidade dos dados.
2. **Relacionamentos entre tabelas**: Em algumas situações, tive dificuldades para identificar corretamente os vínculos entre as tabelas de dimensões e fatos. Após revisar o modelo e consultar a documentação, consegui ajustar os **INNER JOIN** e as **consultas** de forma adequada.
3. **Performance das consultas**: Quando tentei realizar consultas mais complexas, com múltiplos **INNER JOIN**, houve uma queda de performance. Por isso, precisei refinar as consultas, simplificando algumas delas para garantir uma execução mais eficiente.
4. **Visualizações**: Embora as visualizações tenham sido feitas corretamente, no começo eu tive dificuldades para ajustar o tipo de gráfico ideal para cada análise. Com a prática, consegui refinar essas escolhas.

### 🛠️ **Trabalhos Futuros e Melhorias**

Embora o MVP tenha atendido aos objetivos iniciais, existem algumas melhorias e extensões que poderiam ser feitas para enriquecer o trabalho:

1. **Melhorar a qualidade dos dados**: Adicionar mais dados reais ou coletar dados sobre outros fatores, como **causas do acidente** e **detalhes dos danos materiais**, pode enriquecer ainda mais a análise.
2. **Expansão do modelo de dados**: Incorporar dados de **vítimas** (se possível) ou informações sobre o **trânsito** no momento do acidente (por exemplo, volume de veículos).
3. **Outras visualizações**: Integrar mapas de calor e **gráficos interativos**, utilizando ferramentas como **Power BI** ou **Tableau**, para enriquecer ainda mais a análise.


### 💡 **Reflexão**

Esse projeto foi muito valioso para aprimorar minhas habilidades em **SQL**, **modelagem de dados**, e **visualização**. Além disso, pude entender melhor os desafios da **qualidade de dados** e como eles impactam as análises e os resultados. A experiência também foi excelente para consolidar o uso do **Databricks** como ferramenta para trabalhar com grandes volumes de dados e realizar transformações complexas.

Ao final, estou confiante de que esse trabalho tem grande valor para o meu portfólio, tanto pela qualidade técnica das consultas e visualizações quanto pela capacidade de fornecer insights práticos para melhorar a segurança no trânsito.

---
